In [69]:
# Imports
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")  # Apply Seaborn's theme

In [74]:
# Framework
from dataclasses import dataclass

@dataclass
class Apartment:
    square_meters: int
    price_per_sq_m_zl: int
    
    @property
    def price(self) -> int:
        return self.square_meters * self.price_per_sq_m_zl
        
@dataclass
class Mortgage:
    pv: int
    years: int
    i: float # APR, f.e. 7%
    m: int = 12 # payment periods per year
    
    @property
    def periods(self) -> int:
        return self.years * self.m
    
    @property
    def pmt(self) -> int:
        i = self.i / 100
        per_period_annuity_factor = self.m / (self.m + i)
        per_year_factor = self.m * (1 - pow(per_period_annuity_factor, self.periods))
        return self.pv * i / per_year_factor
            
    @property
    def fv(self) -> int:
        return self.pmt * self.periods

class Costs:
    def __init__(
            self, 
            apt: Apartment, mortgage: Mortgage, 
            realtor_rate: int,
            bank_provision_rate: int, bank_promise_zl: int, 
            unexpected_costs_zl: int):
        self.notary_zl = 1010 + 0.004 * (apt.price - 60000)
        self.realtor_costs = apt.price * realtor_rate / 100
        self.bank_provision_zl = mortgage.pv * bank_provision_rate / 100
        self.bank_promise_zl = bank_promise_zl
        self.unexpected_costs_zl = unexpected_costs_zl
        self.mortgage_register_zl = 100 + 200 + 200 + 250
        
    @property
    def all_zl(self) -> int:
        return self.notary_zl + self.realtor_costs + self.bank_provision_zl + self.bank_promise_zl + self.unexpected_costs_zl + self.mortgage_register_zl
    
    



In [75]:
# Logic 

apt = Apartment(
    square_meters=75,
    price_per_sq_m_zl=17500,
)

down_payment=apt.price * 10/100

# TODO. Add support for variable mortgages rates
# TODO. Add support for change of mortgage rate type every N=5 years
mortgage = Mortgage(
    pv = apt.price - down_payment,
    i=7.5,
    years=20,
)

# TODO. Add insurances, etc
costs = Costs(
    apt=apt,
    
    mortgage=mortgage,
    realtor_rate=2,
    bank_provision_rate=2,
    bank_promise_zl=150,
    unexpected_costs_zl=150
)

day_zero_costs = down_payment + costs.all_zl

print(f"Apt. price: {apt.price} for {apt.square_meters} sq.m., {apt.price_per_sq_m_zl} zl/m2")
print(f"Down payment: {down_payment:.2f}")
print(f"Day zero down payment + all mortgage costs: {day_zero_costs:.2f}")
print(f"Overpay on day 0: {day_zero_costs - down_payment:.2f}")

print("\n")

print(f"PMT: {mortgage.pmt:.2f}")
print(f"Loan PV: {mortgage.pv:.2f}")
print(f"Loan FV: {mortgage.fv:.2f}")

print("\n")

total_fv = mortgage.fv + day_zero_costs
total_overpay = total_fv - apt.price
print(f"FV Total: {total_fv:.2f}")
print(f"Total overpay: {total_overpay:.2f}")

Apt. price: 1312500 for 75 sq.m., 17500 zl/m2
Down payment: 131250.00
Day zero down payment + all mortgage costs: 188195.00
Overpay on day 0: 56945.00


PMT: 9516.07
Loan PV: 1181250.00
Loan FV: 2283856.70


FV Total: 2472051.70
Total overpay: 1159551.70


In [76]:
# Comparing against expected inflation

root_year = 2024
expected_inflation_avg = 4.2 / 100
expected_current_monthly_rent = 7000

expected_inflation = pd.Series(
    index=[x for x in range(root_year, root_year + mortgage.years)],
    data=[pow(1 + expected_inflation_avg, x - root_year) for x in range(root_year, root_year + mortgage.years)]
)


monthly_expenditure = pd.DataFrame(
    data=[
        [
            expected_inflation.iloc[i,] * expected_current_monthly_rent,
            mortgage.pmt,
            expected_inflation.iloc[i,] * expected_current_monthly_rent - mortgage.pmt
        ] for i in range(0, expected_inflation.size)
    ],
    columns=["rent", "mortgage pmt", "rent_minus_pmt"],
    index=[i + root_year for i in range(0, expected_inflation.size)]
)
monthly_expenditure.index.name="year"
print("Monthly expenditure")
display(monthly_expenditure)

yearly_expenditure = monthly_expenditure * 12
print("Yearly expenditure")
display(yearly_expenditure)

sell_price = apt.price * expected_inflation.iloc[-1] * 0.8
profit = sell_price - mortgage.fv - day_zero_costs + sum(yearly_expenditure.rent_minus_pmt)

print(f"Sell price - mortgage - downtime pmt + money saved on no rent: {profit}")
yield_rate = (total_fv+profit) / total_fv / mortgage.years 
print(f"Yield rate, avg yearly: {yield_rate * 100:.2f}%")

print(f"WARNING! Next logic is highly questionable. I dunno how to calculate average i considering PV as day zero costs and PMT monthly costs")
compounded_yield_rate = (pow(1 + yield_rate * mortgage.years, 1/mortgage.periods) - 1) * mortgage.m
print(f"Yield rate, compound, {mortgage.m} periods per year: {compounded_yield_rate * 100:.2f}%")


Monthly expenditure


,rent,mortgage pmt,rent_minus_pmt
year,,,
2024,7000.000000,9516.069599,-2516.069599
2025,7294.000000,9516.069599,-2222.069599
2026,7600.348000,9516.069599,-1915.721599
2027,7919.562616,9516.069599,-1596.506983
2028,8252.184246,9516.069599,-1263.885353
2029,8598.775984,9516.069599,-917.293615
2030,8959.924576,9516.069599,-556.145023
2031,9336.241408,9516.069599,-179.828191
2032,9728.363547,9516.069599,212.293948


Yearly expenditure


,rent,mortgage pmt,rent_minus_pmt
year,,,
2024,84000.000000,114192.835186,-30192.835186
2025,87528.000000,114192.835186,-26664.835186
2026,91204.176000,114192.835186,-22988.659186
2027,95034.751392,114192.835186,-19158.083794
2028,99026.210950,114192.835186,-15166.624236
2029,103185.311810,114192.835186,-11007.523376
2030,107519.094906,114192.835186,-6673.740280
2031,112034.896892,114192.835186,-2157.938293
2032,116740.362562,114192.835186,2547.527376


Sell price - mortgage - downtime pmt + money saved on no rent: 92436.9348609125
Yield rate, avg yearly: 5.19%
WARNING! Next logic is highly questionable. I dunno how to calculate average i considering PV as day zero costs and PMT monthly costs
Yield rate, compound, 12 periods per year: 3.56%
